# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

**Antes de empezar:**
- [Instalar spaCy y el modelo de lenguaje con el que van a trabajar](https://spacy.io/models#quickstart)

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

**Recomendación:**
- Hay muchos ejemplos de código en internet, no se sientan obligados a implementar todo desde cero.

### Librerías

In [ ]:
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install sklearn

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso a APIs

In [1]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch
import seaborn as sns
import tqdm
import plotly.express as px
import plotly.graph_objects as go

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# es = es_core_web_sm
nlp = spacy.load("es") # completar con el modelo que van a utilizar

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
musixmatch = Musixmatch('1aa5272f4402bf2f082ad2f3958c2c62') # se puede reemplazar por otra API si da mejores resultados

In [2]:
sns.set_context(context='paper')

### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

# Dataset

## Music Features

In [3]:
#Aux funcs
def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        audio_features = sp.audio_features(track["song_id"])
        track["audio_features"] = audio_features[0]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]

def get_genres(artists_id):
    res = [genres_by_artist_id(x) for x in artists_id if genres_by_artist_id(x)!= []]
    if res != []:
        return res[0]
    return res

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] = track['name']
    row["song_id"] = track['id']
    row["artists"] = [x["name"] for x in track["artists"]]
    row["artists_id"] = [x["id"] for x in track["artists"]]
    row["album_name"] = track['album']['name']
    row["album_id"] = track['album']['id']
    row["audio_features"] = audio_features[0]
    row["genres"] = get_genres(row["artists_id"])
    songs.append(row)
    return songs


In [5]:
PLAYLIST_ID = "2IuD0qZb14cji5y52crdsO"
TEST_PLAYLIST_ID = "3gLmPh92AyeYDKYLaNC8uv"
songs = []

def add_songs_of_playlist(playlist_id,songs_array):

    print("This may take a while...")
    offset = 0
    playlist = sp.playlist_tracks(playlist_id,offset=offset,limit=100)
    batches = playlist["total"] // 100
    print("...downloading "+ str(playlist["total"]) + " songs")
    print("in "+str(batches)+ " batches")
    for j in tqdm.tqdm(range(batches)):
        for i in range(len(playlist["items"])):
            add_track(playlist["items"][i]["track"]["id"],songs_array)
        offset += len(playlist["items"])
        playlist = sp.playlist_tracks(playlist_id, offset=offset,limit=100)
    return

add_songs_of_playlist(PLAYLIST_ID,songs)

This may take a while...


  0%|          | 0/11 [00:00<?, ?it/s]

...downloading 1190 songs
in 11 batches


100%|██████████| 11/11 [19:52<00:00, 108.41s/it]


In [6]:
songs_original_df = pd.DataFrame(songs)
songs_original_df.sample(3)

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
267,1xP7EDeNa518tfq8vxT4cT,Conociendo Rusia,[Conociendo Rusia],[79R7PUc6T6j09G8mJzNml2],"{'danceability': 0.572, 'energy': 0.828, 'key'...","[argentine alternative rock, argentine indie]",43JRoaGJFTTNXCY1FNo9tA,Juro
912,5QqFpppnhRlNXjHlDiDexl,Los De Atrás Vienen Conmigo,"[Calle 13, Café Tacvba]","[0yNSzH5nZmHzeE2xn6Xshb, 09xj0S68Y1OU1vHMCZAIvz]","{'danceability': 0.573, 'energy': 0.856, 'key'...","[latin, latin afrobeat, latin alternative, lat...",5iqhyXGD2Ag4ZeLNyrffPv,No Hay Nadie Como Tú (feat. Café Tacuba)
65,6UXLhItDATUw5fncqIdTSW,El Sonido Original,[Los Palmeras],[2Htm0q72SjGSmsy8EOIhRG],"{'danceability': 0.671, 'energy': 0.807, 'key'...","[cumbia pop, cumbia santafesina, cumbia villera]",3m6havHCZHEHBOG6E10nF5,Cumbia Sobre el Mar


Checking dataframe's consistency.

In [7]:
songs_original_df[[x == [] for x in songs_original_df["genres"]]]

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
48,59tn7tvd1M5XNWwV3TaVWC,"Enrique Santos Discepolo ""El poeta del tango"" ...",[Enrique Santos Discépolo],[0aPYs7yoiP2NtS5xNZXKjg],"{'danceability': 0.492, 'energy': 0.541, 'key'...",[],3PI0FE7JUmEmEyN5YgKPZA,Cambalache
478,0EJRlYjvVcym9K4wrww9vB,"Narcos, Vol. 2 (More Music from the Netflix Or...",[Alonso y Bernardo],[5sskVxLnToHrnwTAICyVF5],"{'danceability': 0.672, 'energy': 0.625, 'key'...",[],1s0ndZpf2KeKEA08CsIFia,Sigue Feliz
498,5mOhH6e2L60jmymgASsbuH,Sobredosis De Chamame,[Amboé],[3CzMaJREWlNNmSe0vtBAKD],"{'danceability': 0.62, 'energy': 0.714, 'key':...",[],6ZbwzxhQ62IoGb70EK7lLg,Sobredosis de Chamamé
526,4xDelBtEq3aJCU8hU6gFLB,Los Mejores Tinkus,[Incas de Oro],[58wFXtpJxfvtigDaRWTNcj],"{'danceability': 0.675, 'energy': 0.477, 'key'...",[],1rzFbkSvxQv6r3PSGjn7Ub,Celia
600,5g5rgxGPlCnRTrYyf173fp,Arteria Ulnar,[Té de Brujas],[39BvzssARgDTZ1Kf0uqNfj],"{'danceability': 0.497, 'energy': 0.651, 'key'...",[],7kGsNBECFyCQ0fBJn2KB6o,Arteria Ulnar
604,43moEeCjsTjk6N25XRin0S,No Le Ganamos a Nadie,[Literal],[0Ec1MqHP5MENR7rK3DtO3G],"{'danceability': 0.568, 'energy': 0.938, 'key'...",[],0F0I189uNvQBdgy1SFNOec,No Le Ganamos a Nadie
606,3F6da9yP7HMGwl88egAqZ5,Contratiempos,[Parientes],[76lUSSvc6Z83CLrIVB7YrE],"{'danceability': 0.549, 'energy': 0.778, 'key'...",[],50GbEo3clyzJRzuAjIFWdz,Contratiempos
613,2fNd57gzWCMwsNVG0K5YQy,Si Tú No Estas (Nashville),[Stokoff],[03wfTeoZex93T5TPxWo3B9],"{'danceability': 0.545, 'energy': 0.652, 'key'...",[],3TCpMjVi4DVzbc5dXLpEeX,Si Tú No Estas (Nashville)
617,6NCW2haZteRywEWZSzc7in,Una Nueva Realidad,[Scones de la Chola],[1n0013t3w2RbIqSYarnPGS],"{'danceability': 0.536, 'energy': 0.571, 'key'...",[],3CQinOLvOg1vMvP9a060xV,Una Nueva Realidad
628,4Bgue5pbIMGEZ61SoULBMr,Epe,[La Extrema Vanguardia],[3p1OOKD3Rs8JsT9I76mACt],"{'danceability': 0.464, 'energy': 0.864, 'key'...",[],48EI8HkseqMBYQw8yl4WL3,Dale!


These cells don't have a genre since Spotify hasn't assigned a genre to their corresponding artists. 
These edge cases will be corrected by hand following these conventions; 

Cambalache's genre is empty. I decided to assign to this the same genres Julio Sosa has.

No nulls in any other column

In [8]:
columns = songs_original_df.columns

for column in columns:
    if songs_original_df[[x == [] for x in songs_original_df[column]]].empty:
        print("No nulls in column ", column)
    else:
        print(">>>>>>> Found nulls in column", column)

No nulls in column  album_id
No nulls in column  album_name
No nulls in column  artists
No nulls in column  artists_id
No nulls in column  audio_features
>>>>>>> Found nulls in column genres
No nulls in column  song_id
No nulls in column  song_name


Let's remove possible duplicated songs

In [19]:
#duplicated_songs = songs_original_df[songs_original_df["song_id"].duplicated(keep='last')]
#duplicated_songs
songs_original_df[songs_original_df.duplicated(subset=["song_id"])==True].count()[0]
#songs_original_df.duplicated(subset=["song_id"],keep='first').any()
#songs_original_df[songs_original_df["song_id"]=="2TNV1bPTWhKTRTVAghIszh"]

2

We find two duplicated tracks of the same song_id in the dataset.
Let's remove them

In [22]:
songs_df = songs_original_df[songs_original_df["song_id"].duplicated(keep='last') != True]
songs_df.duplicated(subset=["song_id"]).any()

False

In [11]:
audio_features_base = pd.DataFrame(list(songs_original_df["audio_features"].values))
audio_features_description = audio_features_base.describe()
audio_features_description

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,1100.000000,1100.000000,1.100000e+03,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,0.223048,0.586041,2.323327e+05,0.711866,0.020731,5.285455,0.232568,-6.543470,0.610000,0.069305,124.442247,3.938182,0.592333
std,0.235524,0.153187,6.468759e+04,0.185329,0.106629,3.540018,0.222429,2.802298,0.487972,0.067027,29.720005,0.313187,0.228801
min,0.000002,0.148000,3.893300e+04,0.108000,0.000000,0.000000,0.027700,-19.575000,0.000000,0.022600,62.850000,1.000000,0.037700
25%,0.028300,0.488000,1.959402e+05,0.593750,0.000000,2.000000,0.095000,-7.887000,0.000000,0.031700,97.955750,4.000000,0.408750
50%,0.139000,0.605000,2.248200e+05,0.744500,0.000002,5.000000,0.138000,-6.102000,1.000000,0.043300,124.939000,4.000000,0.609500
75%,0.346500,0.698000,2.600032e+05,0.863000,0.000223,9.000000,0.296250,-4.610000,1.000000,0.075750,145.991000,4.000000,0.779000
max,0.982000,0.945000,1.129160e+06,0.995000,0.944000,11.000000,0.991000,-0.767000,1.000000,0.514000,204.498000,5.000000,0.976000


Preeliminary analysis of the dataset

In [12]:
#more aux functions
def track_by_feature(feature, value):
    track_id = audio_features_base[audio_features_base[feature]==value]['id']
    track_id = track_id.values.item(0)
    return songs_original_df[songs_original_df['song_id']== track_id]
#example use: 
#print(track_by_feature("valence",0.039100))
#track_by_feature("speechiness",0.492000)

def songs_of_description(statistic):
    row = audio_features_description.loc[statistic]
    keys = row.keys()
    tracks_of_row = []
    for key in keys:
        track = track_by_feature(key,row[key]).to_dict()['song_name']
        track = list(track.values())[0]
        tracks_of_row.append({key: track})
    return tracks_of_row
#example use
#songs_of_description("min")

songs_of_description("max")

[{'acousticness': 'Que Cruz La Que Lleva El Viento'},
 {'danceability': 'Salud y Vida'},
 {'duration_ms': 'Popurrí - En Vivo'},
 {'energy': 'Boom Boom'},
 {'instrumentalness': 'Caballo negro'},
 {'key': 'Robo un Auto'},
 {'liveness': 'Amor de mañana'},
 {'loudness': 'Quién Se Tomó Todo el Vino - En Vivo'},
 {'mode': 'Sin Rumbo'},
 {'speechiness': "Pa'l Norte (feat. Orishas)"},
 {'tempo': 'Como Vas a Hacer'},
 {'time_signature': 'Desafío'},
 {'valence': 'Rock del Gato'}]

In [56]:
songs_of_description("min")

[{'acousticness': 'Porque Hoy Nací'},
 {'danceability': 'Piel'},
 {'duration_ms': 'Luli'},
 {'energy': 'Te Quiero, Te Espero'},
 {'instrumentalness': 'Anoche'},
 {'key': 'El Pibe Tigre'},
 {'liveness': 'Armas Gemelas'},
 {'loudness': 'Outropía'},
 {'mode': 'Desafío'},
 {'speechiness': 'No Tengo'},
 {'tempo': 'Moraleja'},
 {'time_signature': 'El Enrosque'},
 {'valence': 'Piel'}]

## Lyrics

In [17]:
songs_df

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
854,5BXYr7iU56rEX24QQPYcjO,Verde Paisaje del Infierno,[Los Piojos],[0SnyKkoyBaB2fG8IJH4xmU],"{'danceability': 0.497, 'energy': 0.952, 'key'...","[argentine rock, latin alternative, latin rock...",2TNV1bPTWhKTRTVAghIszh,Media Caña
867,1ZUWRmH8qVKhcps3MjlLo5,3er Arco,[Los Piojos],[0SnyKkoyBaB2fG8IJH4xmU],"{'danceability': 0.66, 'energy': 0.258, 'key':...","[argentine rock, latin alternative, latin rock...",6up6cF609S8wsSkDX0egNE,Gris


Decisión de diseño
El género es una lista de géneros.

Hay canciones que no tienen un género asociado, en ese caso la api devuelve un []. Cómo tratar ese caso?

Yo propongo usar el género del artista. 

Qué pasa si shakira y metallica colaboran? qué género le ponés? Una decision posible: los géneros de los dos.

distribución de los géneros en el dataset.

In [45]:
import requests
from collections import Counter
from nltk import ngrams, bigrams
import nltk
import itertools
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import spacy

# aux methods

def song_url_for_request(artist, song_title):
    return "https://api.lyrics.ovh/v1/" + artist + '/' + song_title #str
    # example use:
    # requests.get(song_url_for_request("Death Grips", "Hacker"))
    

def lemmafy(doc):
    lemmas = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
             lemmas.append(token.lemma_)
    return lemmas

def add_lyrics_to_data(all_songs):
    for song in tqdm.tqdm(all_songs):
        try:
            response = requests.get(song_url_for_request(song["artists"][0],song["song_name"])).json()
            json_data = response #json.loads(response.content)
            lyrics_raw = json_data["lyrics"]
            doc = nlp(lyrics_raw)
            lemmed = lemmafy(doc)
            song["lyrics"]=[word.lower() for word in lemmed]
        except: #found a song without lyrics!
            print(song["song_name"]+ ">>>No lyrics")
            song["lyrics"]=None
    return all_songs

In [46]:
all_songs = songs_df.copy()
all_songs = all_songs.T.to_dict().values()
all_songs = add_lyrics_to_data(all_songs)
#i=0
#for song in all_songs:
#    i+=1
#    print(song["artists"][0])
#    if i > 10:
#        break
#all_songs

  1%|▏         | 14/1098 [00:17<22:43,  1.26s/it]

Toro y Pampa - En Vivo>>>No lyrics


  2%|▏         | 20/1098 [00:24<21:48,  1.21s/it]

Perro Amor Explota - Live In Buenos Aires / 2001>>>No lyrics


  2%|▏         | 24/1098 [00:32<29:01,  1.62s/it]

...

Loro Volvé a Tu Jaula>>>No lyrics


100%|█████████▉| 1097/1098 [28:26<00:01,  1.97s/it]

Solo Aspirina>>>No lyrics


100%|██████████| 1098/1098 [28:28<00:00,  1.56s/it]

Sufre Cheto>>>No lyrics


In [47]:
songs_base = pd.DataFrame(all_songs)
len(all_songs)


1098

In [50]:
# sanitize
bool_series = pd.notnull(songs_base["lyrics"])
songs_base = songs_base[bool_series]
songs_base.sample(3)

,album_id,album_name,artists,artists_id,audio_features,genres,lyrics,song_id,song_name
744,2Z1OKCVyqJ1lMHTU612E07,No Hay Imposibles,[Chayanne],[1JbemQ1fPt2YmSLjAFhPBv],"{'danceability': 0.557, 'energy': 0.523, 'key'...","[latin, latin pop, puerto rican pop, tropical]","[siento, palabra, llegar, a, milimetro, y, esf...",5BHOvdkbRanV1tIWaxKSvR,Siento
850,4SiLNaIeJ75aIJz13HjLZr,Esto Es Estopa (Directo Acústico),"[Estopa, Vicentico]","[5ZqnEfVdEGmoPxtELhN7ai, 25THA9HUHoxpCT4LBp7UsZ]","{'danceability': 0.665, 'energy': 0.907, 'key'...","[rumba, spanish pop]","[fuí, a, orillar, reír, y, ver, estar, vi, hab...",5WYAUge1BJ98gSmxH9AbzG,Tu Calorro (feat. Vicentico) - Directo Acústico
7,5IyVKrEhTZhhTbUsB8xdl6,Mundo Guanaco,[Almafuerte],[6qYd7xlmeeeDkPfx6mZ9PV],"{'danceability': 0.577, 'energy': 0.85, 'key':...","[argentine heavy metal, argentine metal, argen...","[mañana, y, achique, pibe, marchar, pedalear, ...",6p5SbKcAnIECU3hoFkZlPQ,El Pibe Tigre


#### Notas del profe

Primero, la letra en sí no se usa, igual que el nombre de la canción. Los n gramas tampoco.

Lo que sí se puede usar son datos obtenidos en base al texto de las canciones, el ejemplo por excelencia es hacer sentiment analysis

Ver
https://elitedatascience.com/python-nlp-libraries


### 2) Elegir tres modelos de clasificadores multiclase

Aquí escogeremos tres modelos diferentes y luego compararemos su rendimiento para esta tarea. El procedimiento será el siguiente:
- Inicializar los modelos
- Entrenarlos usando el split **train** de los datos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 3) Informe: Comparar el rendimiento de los modelos

Una vez entrenados los tres modelos, compararemos su rendimiento:
- Correr los modelos usando el split **test**
- Obtener el reporte de clasificación y la matriz de confusión para cada modelo
- Graficar llevando a 2 dimensiones nuestro split **test** pintando con colores diferentes según la etiqueta correspondiente.
- Graficar de manera similar los resultados obtenidos con cada clasificador y sobre esto la función de clasificación obtenida.
- Guardar los modelos usando **pickle**
- Discutir los resultados obtenidos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 4) Tareas adicionales:

Estas tareas servirán para extrapolar un poco el trabajo básico, y también sumarán puntos extra. Deben elegir una o más de las siguientes:
-  Análisis sobre el balance de clases del dataset, balanceo usando **subsampling** u **oversampling** y comparación de resultados vs el modelo básico
- Optimización de hiperparámetros y comparación de resultados vs el modelo básico
- Graficar importancia de features
- Graficar correlación de features

**Recomendación:**
- Hacer varias ahora puede ahorrarles tiempo en el futuro